KDE-based empirical prediction of melt/nomelt with tas
===
Currently (5/15/19) using adjusted CESM LE data.
---

Global module imports
---

In [1]:
# %matplotlib inline
# %config InlineBackend.print_figure_kwargs = {'bbox_inches':'tight'}
# from plotUtils import PlotUtils

from netCDF4 import Dataset
from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
from scipy import stats
import os
# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# from matplotlib.lines import Line2D
# from matplotlib.text import Text

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet
from itertools import izip

pd.options.mode.chained_assignment = None

Function: define upper/lower temperature limits
---

In [2]:
def setLimits(  mms, var, src = "grid" ):
    '''
        bounds based on KDE of various datasets, 1996-2005.
        tasmax values are STALE and based on ERAI analysis, not CESM LE (5/15/19).
        note: this is a doubly-nested dictionary!
    '''
    bounds = { \
                'tas': { \
                   'aws': { \
                       '06':  ( -8.2,-0.6 ), \
                       '07':  ( -7.6,0.0 ), \
                       '08':  ( -9.9,-0.9 ), \
                       'JJA': ( -8.1,-0.5 ) \
                          }, \
                   'grid': { \
                       '06':  ( -9.1,-1.7 ), \
                       '07':  ( -8.4,-0.9 ), \
                       '08':  ( -10.7,-2.8 ), \
                       'JJA': ( -9.0,-1.5 ) \
                           } \
                       }, \
                'tasmax': { \
                  'aws': { \
                      '06':  ( -2.9, 1.2 ), \
                      '07':  ( -2.1, 1.4 ), \
                      '08':  ( -3.8, 1.4 ), \
                      'JJA': ( -2.8, 1.3 ) \
                         }, \
                  'grid': { \
                      '06':  ( -3.7, 0.9 ), \
                      '07':  ( -2.5, 1.0 ), \
                      '08':  ( -4.5, 1.9 ), \
                      'JJA': ( -3.3, 1.0 ) \
                          } \
                      } \
                 }
    try:
        return bounds[var][src][mms]
    except:
        print "setLimits: Key error(s)!  Args: var =",var,", src =",src,", mms =",mms
        return (None, None)

Function: Predict melt from temperature
---

In [3]:
def predictMelt( M, lwr, upr, npts = 30 ):
    """ predict melt from temperature using empirical transition function"""

    # setup
    melt = np.zeros_like( M ).astype(np.float32)
    
    # set obvious melt points
    flg = M >= upr
    melt[flg] = 1

    # interpolate ambiguous melt points
    flg = (M >= lwr) & (M < upr)
    x = (M[flg] - lwr) / (upr - lwr)
    melt[flg] = x
    
    return melt

Global data
---

In [4]:
# key data
var = "tas"
# var = "tasmax"

model = "cesmle"
# branch = "historical"
branch = "rcp85"
src = "grid"
calibYears = "1996-2005"

# mon = 6
# mon = 7
# mon = 8
mon = "JJA"

# suff = None
suff = "adj"

ens = 1
# 
# ------------------------------------
# derived data
if branch == "historical":
    yr1 = "1996"
    yr2 = "2005"

if branch == "rcp85":
    yr1 = "2071"
    yr2 = "2080"

try:
    mms = "%02d" % mon
except:
    mms = mon

try:
    enss = "%03d" % ens
except:
    enss = ens
    
yrs = yr1+"-"+yr2

File templates etc
---

In [5]:
modelMetaFN = "wrf_geog.nc"

# modelDataDirTemplate = "/Volumes/sbp1/model/pwrf/gis_%s/%s/wrf/postproc/tas/%%03d" % (model, branch )
modelDataDirTemplate = "/Volumes/sbp1/model/pwrf/gis_%s/%s/wrf/postproc/tas/%%s" % (model, branch )
if suff is None:
#     modelDataTemplate = "tas_wrf_%s_%%03d_%s_%s_d.nc" % ( model, yrs, mms )
    modelDataTemplate = "tas_wrf_%s_%%s_%s_%s_d.nc" % ( model, yrs, mms )
    newNCFNTemplate = "wrf_%s_%%s_tas_newmelt_%s_%s.nc" % ( model, yrs, mms )
else:
#     modelDataTemplate = "tas_wrf_%s_%%03d_%s_%s_d_%s.nc" % ( model, yrs, mms, suff )
    modelDataTemplate = "tas_wrf_%s_%%s_%s_%s_d_%s.nc" % ( model, yrs, mms, suff )
    newNCFNTemplate = "wrf_%s_%%s_tas_newmelt_%s_%s_adj.nc" % ( model, yrs, mms )

Load temperature data
----

In [6]:
modelDataDir = modelDataDirTemplate % enss
modelDataFN = modelDataTemplate % enss
newNCFN = newNCFNTemplate % enss

print "Input directory is "+modelDataDir
print "Input file is "+modelDataFN
print "Output file is "+newNCFN

D = Model(None, modelDataDir+"/"+modelDataFN)

time, timeCF = D.loadData( "time" )
print time[0], time[-1]
# dfTime = pd.Series( time, name="Time")

X = np.array( D.loadData( var ) )
nRec, nLat, nLon = X.shape
print "Shape:", nRec, nLat, nLon

Input directory is /Volumes/sbp1/model/pwrf/gis_cesmle/rcp85/wrf/postproc/tas/001
Input file is tas_wrf_cesmle_001_2071-2080_JJA_d_adj.nc
Output file is wrf_cesmle_001_tas_newmelt_2071-2080_JJA_adj.nc
2071-06-01 10:30:00 2080-08-31 10:30:00
Shape: 920 219 197


Load Mote ice sheet mask
---

In [7]:
fn = "icesheet_mask.nc"
ncfn = Dataset(fn, "r")
icemask2d = np.array( ncfn.variables[ "icemask" ] )
icemask3d = np.repeat( icemask2d[None,:,:],nRec,axis=0)

Predict melt, then mask to ice sheet
---

In [8]:
lwr, upr = setLimits( mms, var, src )
print "Lower", lwr, "Upper", upr

meltNewF = predictMelt( X, lwr, upr )

noice = (icemask3d < 1)
meltNewF[noice] = np.nan

nRec2, nLat2, nLon2 = meltNewF.shape
print "Shape:", nRec2, nLat2, nLon2

Lower -9.0 Upper -1.5
Shape: 920 219 197


Convert from 0-1 to 0/1, i.e., turn floats into integers
---

In [9]:
# set critical value for melt/nomelt and _FillValue for integer version
pCrit = 0.5        # default, not expected to be used
if var == "tas":
    pCrit = 0.779
# if var == "tasmax":
#     pCrit = 0.839
intFillValue = -999
print "Critical value =",pCrit,", _FillValue =", intFillValue

# adjust based on critical value
meltNewRnd = np.where( meltNewF < pCrit, 0., 1. )

# convert to integer and retain missing values
nanix = np.isnan(meltNewF)
meltNewI = np.rint( meltNewRnd ).astype(np.int16)
meltNewI[nanix] = intFillValue  # can't use np.nan since that's only for floats!

Critical value = 0.779 , _FillValue = -999


/Library/Python/2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in less


Create a netCDF file for the new data
-----

In [10]:
# new file will have TWO melt variables:
# 1. meltProb = melt prediction as floating point probability
# 2. melt     = melt prediction as 0/1 integer based on critical probability cutoff value
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN, mode='w', format = 'NETCDF4_CLASSIC') 
ncfile.title = "Predicted melt occurrence"
ncfile.bounds = "%.2f, %.2f" % (lwr, upr)
ncfile.source_model = model
ncfile.source_data = src
ncfile.branch = branch
ncfile.period = yrs
ncfile.month = mms
ncfile.calibYears = calibYears
ncfile.predictorVar = var

time_dim = ncfile.createDimension('time', None)
sn_dim  = ncfile.createDimension('south_north', nLat)
we_dim  = ncfile.createDimension('west_east', nLon)

timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis
timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

meltVar1 = ncfile.createVariable('meltProb', np.float32, ('time','south_north','west_east'), \
                                fill_value = np.nan )
meltVar1.description = "Melt occurrence (probability)"
meltVar1.long_name = "Melt occurrence probability (0-1) predicted from near-surface temperature"
meltVar1.critValue = pCrit
meltVar1[:] = meltNewF

meltVar2 = ncfile.createVariable('melt', np.int16, ('time','south_north','west_east'), \
                                fill_value = intFillValue )
meltVar2.description = "Melt occurrence (0/1)"
meltVar2.long_name = "Melt occurrence (0/1) predicted from near-surface temperature"
meltVar2.critValue = pCrit
meltVar2[:] = meltNewI

ncfile.close()